1. 데이터 로드
2. 텍스트 분할
3. 인덱싱(임베딩 후 인덱싱)
4. 검색(retrieval)

In [1]:
# # Data Loader - 웹페이지 데이터 가져오기
# from langchain_community.document_loaders import WebBaseLoader

# url = 'https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EC%A0%95%EC%B1%85%EA%B3%BC_%EC%A7%80%EC%B9%A8'
# loader = WebBaseLoader(url)

# docs = loader.load()

# print(len(docs))
# print(len(docs[0].page_content))
# print(docs[0].page_content[5000:6000])


# # ---------------------------------------------------------------------------------------


# # Text Split (Documents -> small chunks: Documents)
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)

# print(len(splits))
# print(splits[0].page_content)
# print(splits[0].metadata)


# # ---------------------------------------------------------------------------------------


# # Indexing (Texts -> Embedding -> Store)
# from langchain_community.vectorstores import Chroma
# from langchain_community.embeddings import HuggingFaceEmbeddings

# embeddings_model = HuggingFaceEmbeddings(
#     model_name='jhgan/ko-sroberta-nli',
#     model_kwargs={'device':device},
#     encode_kwargs={'normalize_embeddings':True},
# )

# vectorstore = Chroma.from_documents(documents=splits,
#                                     embedding=embeddings_model)

# docs = vectorstore.similarity_search("격하 과정에 대해서 설명해주세요.")
# print(len(docs))
# print(docs[0].page_content)


In [1]:
# Data Loader - 웹페이지 데이터 가져오기
import json
import pymupdf4llm
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document

from langchain_community.vectorstores import Chroma, FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.llms import Ollama

In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from IPython.display import Markdown

pdf_filepath = 'datasets/여비산정기준표.pdf'

md_text = pymupdf4llm.to_markdown(pdf_filepath) 

# Jupyter Notebook에서 마크다운 표로 렌더링하여 출력
display(Markdown(md_text))

|구 분|운 임|Col3|Col4|Col5|일 비<br>(1일당)|숙박비<br>(1야당)|식 비<br>(1일당)|
|---|---|---|---|---|---|---|---|
|구 분|철 도|선 박|항 공|자동차|자동차|자동차|자동차|
|교 수<br>부 교 수<br>5급(부참사) 이상<br>사무직원|KTX<br>(특실)|2 등<br>정 액|〃|〃|￦30,000|￦100,000|￦40,000|
|조 교 수<br>전 임 강 사<br>일 반 직 원|KTX<br>(보통)|〃|〃|〃|￦30,000|￦100,000|￦40,000|
|조 교<br>사무보조원<br>일 용 직|KTX<br>(보통)|〃|〃|〃|￦20,000|￦80,000|￦30,000|




|현행 지역 구분|세부 지역 구분|교통비(원)|
|---|---|---|
|인천시내|강화군, 옹진군 및 도서지역 제외한 인천시내|10,000|
|서울,안양,과천 외|서울,안양,과천,군포,의왕,안산,시흥,광명,부천,김포 外|20,000|
|수원,분당,일산 외|수원,분당,일산,강화,영종도,화성 外(인천,서울 지역을 제외한 수도권 일원)|25,000|










|구 분|Col2|교 통 비|체 재 비|Col5|Col6|비 고|
|---|---|---|---|---|---|---|
|구 분|구 분|교 통 비|일 비<br>(1일당)|숙박비<br>(1야당)|식 비<br>(1일당)|식 비<br>(1일당)|
|교 수<br>부 교 수<br>부참여(3급)<br>이 상|가|항공임 2등 운임 실비|$35|$220|$110|가 : 스위스,이스라엘, 아이슬<br>란드,노르웨이, 덴마크,<br>아일랜드,오스트레일리아,<br>뉴질랜드,룩셈부르크,스웨<br>덴, 영국, 핀란드, 캐나다,<br>미국, 네덜란드,벨기에,<br>일본, 오스트리아, 프랑스,<br>독일,이탈리아, 홍콩,싱가포르,<br>러시아(모스크바), 중국(베이징,<br>상하이,선전,광저우)<br>나 : “가”지역을 제외한 지역|
|교 수<br>부 교 수<br>부참여(3급)<br>이 상|나|〃|$35|$160|$80|$80|
|조 교 수<br>전임강사<br>참사(4급)<br>이 하|가|항공임 3등 운임 실비|$30|$200|$85|$85|
|조 교 수<br>전임강사<br>참사(4급)<br>이 하|나|〃|$30|$150|$60|$60|




In [4]:
# PDF를 마크다운 형식으로 변환 (페이지별 청크 포함)
md_text = pymupdf4llm.to_markdown(pdf_filepath)

# 2. 마크다운 → 텍스트 청크 분할
splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=50) #잘 안돼면 512, 768, 1024 등으로 변경
chunks = splitter.create_documents([md_text])

# 3. Document 객체 생성
documents = [Document(page_content=chunk.page_content, metadata={"source": pdf_filepath}) for chunk in chunks]

embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-m3',
    model_kwargs={'device':device},
    encode_kwargs={'normalize_embeddings':True},
)

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_template("""
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question.
If the user asks for a simple answer, summarize the key points.
If the question is unrelated to the context in the regulations, respond with "관련 정보를 찾을 수 없습니다."
You must answer in Korean.

#Context: 
{context}

#Question:
{question}

#Answer:
""")

C:\Users\minja\AppData\Local\Temp\ipykernel_8508\115859068.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


In [5]:
# llm = Ollama(model="llama3.2:3b")
# llm = Ollama(model="deepseek-r1:1.5b")
llm = Ollama(model="gemma3:4b")
# llm = ChatOpenAI(model_name = "gpt-4o-mini", temperature=0)

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\minja\AppData\Local\Temp\ipykernel_8508\4235443742.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:4b")


In [6]:
question = "공무형평상 부득이한 경우는?"
result = rag_chain.invoke(question)
print(result)

관련 정보를 찾을 수 없습니다.
